In [12]:
import tensorflow as tf
print(tf.__version__)
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(
housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
X_train_full, y_train_full)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

from tensorflow import keras

input_ = keras.layers.Input(shape = X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation='relu', name = 'hidden1')(input_)
hidden2 = keras.layers.Dense(30, activation='relu', name='hidden2')(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)

model = keras.Model(inputs = [input_], outputs= [output])


2.2.0


In [13]:
model.compile(optimizer = 'sgd', loss= 'mse')

In [14]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('mymodel.h5', save_best_only=True)

In [15]:
model.fit(X_train,y_train, validation_data = (X_valid,y_valid), epochs=5, callbacks=[checkpoint_cb])

Epoch 1/5
363/363 [==============================] - 1s 2ms/step - loss: 1.0026 - val_loss: 0.5901
Epoch 2/5
363/363 [==============================] - 1s 1ms/step - loss: 0.7388 - val_loss: 0.7533
Epoch 3/5
363/363 [==============================] - 1s 2ms/step - loss: 0.8907 - val_loss: 0.4757
Epoch 4/5
363/363 [==============================] - 1s 1ms/step - loss: 4.6909 - val_loss: 0.4843
Epoch 5/5
363/363 [==============================] - 1s 1ms/step - loss: 0.6601 - val_loss: 0.4508


In [16]:
model = keras.models.load_model('mymodel.h5')

In [19]:
y_pred =model.predict(X_test)

In [21]:
import numpy as np
np.mean(keras.losses.mean_squared_error(y_test, y_pred))

2.1910644

In [22]:
#using EarlyStopping : it interrupts the training, when there is no  improvements on validation loss for a number of epochs defines by
#patience parameter

# We can use both checkpoint as well as early stopping callbacks to store the best model and interrrupts the training when there is no
#improvement

earlyStopping = keras.callbacks.EarlyStopping(patience=10, restore_best_weights= True)
#restore best Weight parameter restores the best weights after the training ends
#there is no need to load_weights again.
model.fit(X_train, y_train, validation_data=(X_valid,y_valid), epochs=100, callbacks=[checkpoint_cb, earlyStopping]) 

Epoch 1/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5033 - val_loss: 0.4279
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4222 - val_loss: 0.3957
Epoch 3/100
363/363 [==============================] - 1s 1ms/step - loss: 0.4197 - val_loss: 0.3909
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4999 - val_loss: 0.3818
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4021 - val_loss: 0.3720
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3909 - val_loss: 0.4208
Epoch 7/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3728 - val_loss: 0.3579
Epoch 8/100
363/363 [==============================] - 1s 1ms/step - loss: 0.3584 - val_loss: 0.3579
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3653 - val_loss: 0.3614
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3469 - val_lo

In [24]:
# we can define our own call backs
class mycallback(keras.callbacks.Callback):
  def on_epoch_end(self, epochs, logs):
    if(logs.get('accuracy')> 0.99):
      print("Reached 99% accuracy")
      self.model.stop_training = True

In [ ]:
callback = mycallback()
model.fit(callbacks = [callback])